In [1]:
import augmentor
import dataprovider3 as dp3
import h5py
import numpy as np
import os

### FIB-25 Validation Sample

In [2]:
home = os.path.expanduser('~')
fib25_path = os.path.join(home, 'Data_local/FIB-25/validation_sample')

In [3]:
fpath = os.path.join(fib25_path, 'img.h5')
with h5py.File(fpath, 'r') as f:
    img = f['/main'][...]

fpath = os.path.join(fib25_path, 'seg.h5')
with h5py.File(fpath, 'r') as f:
    seg = f['/main'][...]
    
fpath = os.path.join(fib25_path, 'msk.h5')
with h5py.File(fpath, 'r') as f:
    msk = f['/main'][...]

In [4]:
img = (img/255.).astype(np.float32)

In [5]:
# DataSet
dset = dp3.Dataset()
dset.add_data('img', img)
dset.add_data('seg', seg)
dset.add_mask('seg_mask', msk, loc=True)

In [27]:
# DataProvider
dims = (240,240,240)
spec = {'img': dims, 'seg': dims, 'seg_mask': dims}
dp = dp3.DataProvider(spec)
dp.add_dataset(dset)
dp.set_imgs(['img'])
dp.set_segs(['seg'])

In [28]:
sample = dp()

In [29]:
sample['img'].shape

(1, 240, 240, 240)

In [30]:
n = 12

In [31]:
projections = []
projections.append(augmentor.NormalView(n))
projections.append(augmentor.TiltedView(n,  1, -1))
projections.append(augmentor.TiltedView(n, -1, -1))
projections.append(augmentor.TiltedView(n,  1, -2))
projections.append(augmentor.TiltedView(n, -1, -2))

In [32]:
%time p0 = projections[0](sample['img'])

CPU times: user 5.75 ms, sys: 1.25 ms, total: 7 ms
Wall time: 6.22 ms


In [33]:
%time p1 = projections[1](sample['img'])

CPU times: user 21.2 ms, sys: 4.61 ms, total: 25.8 ms
Wall time: 24.8 ms


In [34]:
%time p2 = projections[2](sample['img'])

CPU times: user 26.9 ms, sys: 1.63 ms, total: 28.5 ms
Wall time: 27.4 ms


In [35]:
%time p3 = projections[3](sample['img'])

CPU times: user 21.4 ms, sys: 630 µs, total: 22.1 ms
Wall time: 21.1 ms


In [36]:
%time p4 = projections[4](sample['img'])

CPU times: user 22.8 ms, sys: 0 ns, total: 22.8 ms
Wall time: 21.6 ms


In [37]:
results = [p(sample['img']) for p in projections]

In [38]:
for r in results:
    print(r.shape)

(1, 20, 240, 240)
(1, 20, 240, 240)
(1, 20, 240, 240)
(1, 20, 240, 240)
(1, 20, 240, 240)


In [44]:
ts = np.concatenate(results, axis=0)

In [46]:
import napari

viewer = napari.view_image((ts*255).astype(np.uint8), name='image')
napari.run()